In [1]:
import pandas as pd
import os
import sys

# Import our custom DQ module
from dq_checks import DQ

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## Run Data Quality Checks

Now let's configure and run the actual DQ checks using our DQ class from dq_checks.py


In [2]:
# Configure the DQ checker
th_values = {
    'val_range': 0,  # Check for values below 0 (negative prices, etc.)
    'time_cross_consistency': 2,  # Minimum occurrences threshold
}

input_tables = {
    'val_range': [('DPS_PRICE', 'PRICE'), ('DPS_PROMO', 'PROMO_PRICE')],
    'cross_consistency': ['DPS_SELL_OUT', 'DPS_PRICE', 'DPS_STOCK'],
    'time_cross_consistency': [['DPS_SELL_OUT', 'DPS_STOCK'], ['DPS_STOCK', 'DPS_SELL_OUT']],
}

lvl_data = {
    'LOCATION': 'DPS_LOCATION',
    'PRODUCT': 'DPS_PRODUCT',
    'CUSTOMER': 'DPS_CUSTOMER',
    'DISTR_CHANNEL': 'DPS_DISTR_CHANNEL'
}

# Create DQ instance
dq = DQ(
    check_id=123, 
    check_name='Supply Chain Data Quality Check', 
    client=666,
    input_tables=input_tables, 
    th_values=th_values, 
    lvl_data=lvl_data,
    data_path='data/'  # Fixed path
)

In [3]:
dq.check()

Starting data quality checks for: Supply Chain Data Quality Check

[1/3] Checking value ranges...
      Found 0 value range issues

[2/3] Checking cross-table consistency...
      Found 1172 cross-consistency issues

[3/3] Checking time-based consistency...
      Found 36297 time-consistency issues

[4/4] Formatting output...

COMPLETE: Found 37469 total issues



In [4]:
dq.data_quality_output

,INPUT_TABLE,WARNING_TYPE,WARNING,PERIOD_DT,INPUT_VALUE,LOCATION_LVL_ID6,LOCATION_LVL,PRODUCT_LVL_ID8,PRODUCT_LVL,CUSTOMER_LVL_ID6,CUSTOMER_LVL,DISTR_CHANNEL_LVL_ID2,DISTR_CHANNEL_LVL
0,DPS_SELL_OUT && DPS_PRICE,cross_consistency,IDs from DPS_SELL_OUT not found in DPS_PRICE,NaN,NaN,600002,6,80001,8,6000015,6,1,2
1,DPS_SELL_OUT && DPS_PRICE,cross_consistency,IDs from DPS_SELL_OUT not found in DPS_PRICE,NaN,NaN,600002,6,80001,8,6000018,6,1,2
2,DPS_SELL_OUT && DPS_PRICE,cross_consistency,IDs from DPS_SELL_OUT not found in DPS_PRICE,NaN,NaN,600004,6,80001,8,6000019,6,1,2
3,DPS_SELL_OUT && DPS_PRICE,cross_consistency,IDs from DPS_SELL_OUT not found in DPS_PRICE,NaN,NaN,600009,6,80001,8,6000020,6,1,2
4,DPS_SELL_OUT && DPS_PRICE,cross_consistency,IDs from DPS_SELL_OUT not found in DPS_PRICE,NaN,NaN,600010,6,80001,8,6000007,6,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37464,DPS_STOCK && DPS_SELL_OUT,time_cross_consistency,2237 records (89.1%) from DPS_STOCK missing in...,28-Mar-21,2.0,600011,6,80074,8,<NA>,6,<NA>,2
37465,DPS_STOCK && DPS_SELL_OUT,time_cross_consistency,2237 records (89.1%) from DPS_STOCK missing in...,4-Apr-21,2.0,600011,6,80074,8,<NA>,6,<NA>,2
37466,DPS_STOCK && DPS_SELL_OUT,time_cross_consistency,2237 records (89.1%) from DPS_STOCK missing in...,11-Apr-21,2.0,600011,6,80074,8,<NA>,6,<NA>,2
37467,DPS_STOCK && DPS_SELL_OUT,time_cross_consistency,2237 records (89.1%) from DPS_STOCK missing in...,18-Apr-21,2.0,600011,6,80074,8,<NA>,6,<NA>,2


## Explicit DQ Check Results

Below we execute the complete data quality check algorithm and review the findings


In [7]:
# Get summary of issues found
summary = dq.get_summary()

print(f"\nTotal Issues Found: {summary['total_issues']}")
print(f"\nIssues by Type:")
for issue_type, count in summary['by_type'].items():
    print(f"  - {issue_type}: {count}")

if 'by_table' in summary:
    print(f"\nTop 5 Table Pairs with Most Issues:")
    sorted_tables = sorted(summary['by_table'].items(), key=lambda x: x[1], reverse=True)[:5]
    for table, count in sorted_tables:
        print(f"  - {table}: {count}")




Total Issues Found: 37469

Issues by Type:
  - time_cross_consistency: 36297
  - cross_consistency: 1172

Top 5 Table Pairs with Most Issues:
  - DPS_SELL_OUT && DPS_STOCK: 34177
  - DPS_STOCK && DPS_SELL_OUT: 2239
  - DPS_SELL_OUT && DPS_PRICE: 537
  - DPS_PRICE && DPS_SELL_OUT: 395
  - DPS_PRICE && DPS_STOCK: 118


In [8]:
# Display sample issues from each category
print("Sample Data Quality Issues:\n")

if not dq.data_quality_output.empty:
    for warning_type in dq.data_quality_output['WARNING_TYPE'].unique():
        print(f"\n{warning_type.upper()} - Sample Records:")
        print("-" * 80)
        sample = dq.data_quality_output[dq.data_quality_output['WARNING_TYPE'] == warning_type].head(3)
        print(sample[['INPUT_TABLE', 'WARNING_TYPE', 'WARNING']].to_string(index=False))
else:
    print("No issues found!")


Sample Data Quality Issues:


CROSS_CONSISTENCY - Sample Records:
--------------------------------------------------------------------------------
              INPUT_TABLE      WARNING_TYPE                                      WARNING
DPS_SELL_OUT && DPS_PRICE cross_consistency IDs from DPS_SELL_OUT not found in DPS_PRICE
DPS_SELL_OUT && DPS_PRICE cross_consistency IDs from DPS_SELL_OUT not found in DPS_PRICE
DPS_SELL_OUT && DPS_PRICE cross_consistency IDs from DPS_SELL_OUT not found in DPS_PRICE

TIME_CROSS_CONSISTENCY - Sample Records:
--------------------------------------------------------------------------------
              INPUT_TABLE           WARNING_TYPE                                                      WARNING
DPS_SELL_OUT && DPS_STOCK time_cross_consistency 34059 records (99.2%) from DPS_SELL_OUT missing in DPS_STOCK
DPS_SELL_OUT && DPS_STOCK time_cross_consistency 34059 records (99.2%) from DPS_SELL_OUT missing in DPS_STOCK
DPS_SELL_OUT && DPS_STOCK time_cross_consist

In [8]:
# Show the full output table structure
print(f"\nFull Data Quality Output Table: {len(dq.data_quality_output)} rows")
print("\nColumn Names:", list(dq.data_quality_output.columns))
print("\nFirst 10 records:")
dq.data_quality_output.head(10)



Full Data Quality Output Table: 37469 rows

Column Names: ['INPUT_TABLE', 'WARNING_TYPE', 'WARNING', 'PERIOD_DT', 'INPUT_VALUE', 'LOCATION_LVL_ID6', 'LOCATION_LVL', 'PRODUCT_LVL_ID8', 'PRODUCT_LVL', 'CUSTOMER_LVL_ID6', 'CUSTOMER_LVL', 'DISTR_CHANNEL_LVL_ID2', 'DISTR_CHANNEL_LVL']

First 10 records:


,INPUT_TABLE,WARNING_TYPE,WARNING,PERIOD_DT,INPUT_VALUE,LOCATION_LVL_ID6,LOCATION_LVL,PRODUCT_LVL_ID8,PRODUCT_LVL,CUSTOMER_LVL_ID6,CUSTOMER_LVL,DISTR_CHANNEL_LVL_ID2,DISTR_CHANNEL_LVL
0,DPS_SELL_OUT && DPS_PRICE,cross_consistency,IDs from DPS_SELL_OUT not found in DPS_PRICE,NaN,NaN,600002,6,80001,8,6000015,6,1,2
1,DPS_SELL_OUT && DPS_PRICE,cross_consistency,IDs from DPS_SELL_OUT not found in DPS_PRICE,NaN,NaN,600002,6,80001,8,6000018,6,1,2
2,DPS_SELL_OUT && DPS_PRICE,cross_consistency,IDs from DPS_SELL_OUT not found in DPS_PRICE,NaN,NaN,600004,6,80001,8,6000019,6,1,2
3,DPS_SELL_OUT && DPS_PRICE,cross_consistency,IDs from DPS_SELL_OUT not found in DPS_PRICE,NaN,NaN,600009,6,80001,8,6000020,6,1,2
4,DPS_SELL_OUT && DPS_PRICE,cross_consistency,IDs from DPS_SELL_OUT not found in DPS_PRICE,NaN,NaN,600010,6,80001,8,6000007,6,1,2
5,DPS_SELL_OUT && DPS_PRICE,cross_consistency,IDs from DPS_SELL_OUT not found in DPS_PRICE,NaN,NaN,600010,6,80001,8,6000010,6,1,2
6,DPS_SELL_OUT && DPS_PRICE,cross_consistency,IDs from DPS_SELL_OUT not found in DPS_PRICE,NaN,NaN,600011,6,80001,8,6000008,6,1,2
7,DPS_SELL_OUT && DPS_PRICE,cross_consistency,IDs from DPS_SELL_OUT not found in DPS_PRICE,NaN,NaN,600012,6,80001,8,6000014,6,1,2
8,DPS_SELL_OUT && DPS_PRICE,cross_consistency,IDs from DPS_SELL_OUT not found in DPS_PRICE,NaN,NaN,600002,6,80002,8,6000015,6,1,2
9,DPS_SELL_OUT && DPS_PRICE,cross_consistency,IDs from DPS_SELL_OUT not found in DPS_PRICE,NaN,NaN,600002,6,80002,8,6000018,6,1,2


### Export Results

Save the data quality output to CSV for further analysis


In [9]:
# Save results to CSV file
output_file = 'data_quality_output.csv'
dq.data_quality_output.to_csv(output_file, index=False)
print(f"Data quality results saved to: {output_file}")


Data quality results saved to: data_quality_output.csv
